In [14]:
import pandas as pd
from datetime import datetime, timedelta
 
df = pd.read_csv('Change hours flights dataset.csv')

# Function to round up to nearest 51 (for departures)
def round_to_nearest_51(time_str):
    if time_str == '24:00':
        time_str = '00:00'
    
    # Convert time to object, define target time and calculate difference
    time = datetime.strptime(time_str, '%H:%M')
    target_time = time.replace(minute=51)
    time_diff = time - target_time
    
    # If the time is before 51 minutes, calculate for previous hour
    if time_diff.total_seconds() < 0:
        prev_target = target_time - timedelta(hours=1)
        if abs(time - prev_target) < abs(time_diff):
            target_time = prev_target
    
    # If after 51 minutes, check if closer to the next hour's 51
    if time_diff.total_seconds() > 0:
        next_target = target_time + timedelta(hours=1)
        if abs(time - next_target) < abs(time_diff):
            target_time = next_target
    
    return target_time.strftime('%H:%M')

# New function for rounding up to nearest 53 (for arrivals)
def round_to_nearest_53(time_str):
    if time_str == '24:00':
        time_str = '00:00'
    
    # Convert time to object, define target time and calculate difference
    time = datetime.strptime(time_str, '%H:%M')
    target_time = time.replace(minute=53)
    time_diff = time - target_time
    
    # If the time is before 53 minutes, calculate for previous hour
    if time_diff.total_seconds() < 0:
        prev_target = target_time - timedelta(hours=1)
        if abs(time - prev_target) < abs(time_diff):
            target_time = prev_target
    
    # If after 53 minutes, check if closer to the next hour's 53
    if time_diff.total_seconds() > 0:
        next_target = target_time + timedelta(hours=1)
        if abs(time - next_target) < abs(time_diff):
            target_time = next_target
    
    return target_time.strftime('%H:%M')

# New columns
df['sched_dep_time (modified)'] = df['sched_dep_time'].apply(round_to_nearest_51)
df['sched_arr_time (modified)'] = df['sched_arr_time'].apply(round_to_nearest_53)

# Reorder columns and insert
cols = df.columns.tolist()
dep_index = cols.index('sched_dep_time') + 1
cols.insert(dep_index, 'sched_dep_time (modified)')
arr_index = cols.index('sched_arr_time') + 1
cols.insert(arr_index, 'sched_arr_time (modified)')

df = df[cols]

# Save modified DataFrame back to a new CSV
df.to_csv('modified_flight_data_correct.csv', index=False)


In [20]:
df = df.sort_values(by=['date' , 'sched_dep_time'])

df = df.reset_index()


In [21]:
df = df.drop(['index'], axis=1)
df.head()

,date,dep_time,sched_dep_time,sched_dep_time (modified),dep_delay,arr_time,sched_arr_time,sched_arr_time (modified),arr_delay,tailnum,air_time,name,sched_dep_time (modified),sched_arr_time (modified)
0,2013-01-01,05:58,06:00,05:51,-2.0,09:24,09:17,08:53,7.0,N29129,345.0,United Air Lines Inc.,05:51,08:53
1,2013-01-01,06:58,07:00,06:51,-2.0,10:27,10:25,10:53,2.0,N627VA,361.0,Virgin America,06:51,10:53
2,2013-01-01,07:02,07:00,06:51,2.0,10:58,10:14,09:53,44.0,N779JB,381.0,JetBlue Airways,06:51,09:53
3,2013-01-01,07:43,07:30,07:51,13.0,11:07,11:00,10:53,7.0,N338AA,358.0,American Airlines Inc.,07:51,10:53
4,2013-01-01,08:29,08:30,08:51,-1.0,11:52,12:00,11:53,-8.0,N554UA,360.0,United Air Lines Inc.,08:51,11:53


In [22]:
df.to_csv('departures.csv', index=False)

In [23]:
df.to_csv('arrivals.csv', index=False)